In [1]:
!pip install yfinance

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%%**********************]  1 of 1 completed


In [4]:
data.shape

(1640, 6)

In [6]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,52.651001,53.799000,52.651001,53.660500,53.598984,31766000
2018-01-03,53.696499,54.805000,53.671501,54.576000,54.513435,31318000
2018-01-04,54.854500,55.203999,54.713001,54.787998,54.725189,26052000


In [7]:
# Sort the data points based on indexes just for confirmation
data.sort_index(inplace = True)

In [41]:
data = data.loc[~data.index.duplicated(keep='first')]

In [8]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,189.899994,190.169998,187.779999,189.029999,189.029999,21035900
2024-07-09,190.309998,191.360001,188.720001,188.979996,188.979996,15121400
2024-07-10,189.149994,191.750000,189.029999,191.179993,191.179993,15937800


In [9]:
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [12]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1640.000000,1640.000000,1640.000000,1640.000000,1640.000000,1.640000e+03
mean,96.431207,97.519717,95.416816,96.505176,96.397213,3.431914e+07
std,35.321469,35.666448,35.012491,35.363074,35.328996,1.541815e+07
min,49.216000,50.605999,48.882999,49.233501,49.177063,9.312000e+06
25%,60.873750,61.342250,60.294374,60.767500,60.697838,2.446700e+07
50%,94.085003,95.544998,92.544998,94.105499,93.997620,3.053000e+07
75%,128.651752,130.293247,127.161499,128.390377,128.243195,3.900922e+07
max,190.309998,191.750000,189.029999,191.179993,191.179993,1.331780e+08


In [13]:
import plotly.graph_objects as go

# Check the trend in Closing Values
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900,
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [14]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900,
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

In [15]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm.notebook import tnrange

In [16]:
# Filter only required data
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,53.660500,31766000
2018-01-03,54.576000,31318000
2018-01-04,54.787998,26052000


In [17]:
# Confirm the Testing Set length
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [18]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length):
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [19]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/1608 [00:00<?, ?it/s]

In [20]:
# Check the shapes
X.shape , Y.shape

((1608, 32, 2), (1608,))

In [21]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [22]:
# Check Training Dataset Shape
Xtrain.shape , Ytrain.shape

((639, 32, 2), (639,))

In [23]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((969, 32, 2), (969,))

In [24]:
# Create a Scaler to Scale Vectors with Multiple Dimensions
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X

In [25]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [26]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [27]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [28]:
# Save your objects for future purposes
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [31]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [32]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
639/639 [==============================] - 1014s 2s/step - loss: 0.0100 - val_loss: 1.4120 - lr: 0.0020
Epoch 2/10
639/639 [==============================] - 968s 2s/step - loss: 0.0115 - val_loss: 1.1913 - lr: 0.0020
Epoch 3/10
639/639 [==============================] - 967s 2s/step - loss: 0.0086 - val_loss: 1.0941 - lr: 0.0020
Epoch 4/10
639/639 [==============================] - 928s 1s/step - loss: 0.0071 - val_loss: 0.9015 - lr: 0.0020
Epoch 5/10
639/639 [==============================] - 955s 1s/step - loss: 0.0069 - val_loss: 0.7807 - lr: 0.0020
Epoch 6/10
639/639 [==============================] - 960s 2s/step - loss: 0.0060 - val_loss: 0.6906 - lr: 0.0020
Epoch 7/10
639/639 [==============================] - 930s 1s/step - loss: 0.0058 - val_loss: 0.6895 - lr: 0.0020
Epoch 8/10
639/639 [==============================] - 968s 2s/step - loss: 0.0054 - val_loss: 0.6646 - lr: 0.0020
Epoch 9/10
639/639 [==============================] - 935s 1s/step - loss: 0.0049 - val

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (1, 32, 1024)             2109440   
 al)                                                             
                                                                 
 lstm_1 (LSTM)               (1, 256)                  1311744   
                                                                 
 dropout (Dropout)           (1, 256)                  0         
                                                                 
 dense (Dense)               (1, 64)                   16448     
                                                                 
 dropout_1 (Dropout)         (1, 64)                   0         
                                                                 
 dense_1 (Dense)             (1, 32)                   2080      
                                                        

In [34]:
model.load_weights("best_weights.h5")

In [35]:
Predictions = model.predict(Xtest)

31/31 [==============================] - 13s 306ms/step


In [36]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [37]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [38]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [39]:
test_df.head()

,Actual,Predicted
Date,,
2020-09-01,82.753998,80.480927
2020-09-02,85.869499,80.833374
2020-09-03,81.475502,81.106369
2020-09-04,79.060501,81.027260
2020-09-08,76.180000,80.859489


In [40]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [42]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [43]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [44]:
Predictions = model.predict(Total_features)

51/51 [==============================] - 17s 337ms/step


In [46]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [47]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [48]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()